In [11]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder

# Local imports
from src.collaborative_filtering import *
#from src.database import * <--- database approach instead of memory usage

# Collaborative Filtering

### We read x amount of playlist slices into memory
Each contains 1000 playlists. For example, 20 slices will take about 5GB of memory

In [3]:
folder = "data/playlist_data/"
co_occurences = update_co_occurrences_from_folder(folder, slice_limit=5)

Processing slice 1/5: mpd.slice.0-999.json
Processing slice 2/5: mpd.slice.1000-1999.json
Processing slice 3/5: mpd.slice.10000-10999.json
Processing slice 4/5: mpd.slice.100000-100999.json
Processing slice 5/5: mpd.slice.101000-101999.json


### We create a playlists for testing recommendations

In [4]:
playlist_1 = [
    ("Smash Mouth" ,"All Star"),
    ("Rick Astley", "Never Gonna Give You Up"),
    ("The Proclaimers", "I'm Gonna Be (500 Miles)"),
    ("Backstreet Boys", "I Want It That Way"),
    ("The Killers", "Mr. Brightside"),
]

# q: what should we name this playlist?
# a: "The Ultimate 90s Playlist"

In [5]:
get_recommendations(playlist_1, co_occurences, top_n=10)

Journey - Don't Stop Believin' - 0.14326923700397048
Dexys Midnight Runners - Come On Eileen - 0.12021304008227918
Queen - Bohemian Rhapsody - Remastered 2011 - 0.10991054384754884
Smash Mouth - All Star - 0.09869261130935814
OutKast - Hey Ya! - Radio Mix / Club Mix - 0.08831940256565037
Rick Springfield - Jessie's Girl - 0.08788703101660567
Vanessa Carlton - A Thousand Miles - 0.08568211967649743
R. Kelly - Ignition - Remix - 0.08495819790551361
Spice Girls - Wannabe - Radio Edit - 0.08369803941053391
Lynyrd Skynyrd - Sweet Home Alabama - 0.08208312930294659


In [6]:
playlist_2 = [
    ("Drake", "One Dance"),
    ("The Chainsmokers", "Closer"),
    ("Ed Sheeran", "Shape of You"),
    ("Justin Bieber", "Sorry"),
    ("The Weeknd", "Starboy"),
]

# q: what should we name this playlist?
# a: "The Ultimate Pop Playlist"

In [7]:
get_recommendations(playlist_2, co_occurences, top_n=10)

Major Lazer - Cold Water (feat. Justin Bieber & MØ) - 0.13594257771317855
The Chainsmokers - Don't Let Me Down - 0.13133278571985335
The Chainsmokers - Closer - 0.1281916074528291
Drake - One Dance - 0.11468946249907858
DRAM - Broccoli (feat. Lil Yachty) - 0.10871439788185103
Mike Posner - I Took A Pill In Ibiza - Seeb Remix - 0.10678774664197961
DJ Snake - Let Me Love You - 0.10555517322945326
Calvin Harris - This Is What You Came For - 0.0995580318367021
The Chainsmokers - Roses - 0.09544866523600537
The Weeknd - Starboy - 0.09179862904217692


___
# Content based filtering

In [12]:
song_data = pd.read_csv('data/song_data.csv')
# Remove key, mode, time_signature, and duration_ms
song_data = song_data.drop(['key', 'mode', 'time_signature', 'duration_ms'], axis=1)

C:\Users\David\AppData\Local\Temp\ipykernel_18884\3785175046.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  song_data = pd.read_csv('data/song_data.csv')


## Implement KNN

#### Feature scaling

In [13]:
# Scale numerical features
numerical_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']
X = song_data[numerical_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

ValueError: could not convert string to float: '-10.5w34'

In [14]:
# Genre encoding
genres = song_data[['genre']]
encoder = OneHotEncoder(sparse=False)
genres_encoded = encoder.fit_transform(genres)#.toarray()

c:\Users\David\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#### Combine genres and numerical features and fit

In [16]:
genre_weight = 20 # The higher the weight, the more important the genre is in the recommendation

X_combined = np.concatenate((X_scaled, genres_encoded * genre_weight), axis=1)

knn = NearestNeighbors(n_neighbors=10, metric='euclidean')
knn.fit(X_combined)

NameError: name 'X_scaled' is not defined

## Song Query

In [ ]:
# Define song and artist which we want to find similar songs for
song_name = 'What Once Was'
artist_name = 'Her\'s'

In [ ]:
# Find the song in the dataset
song_query = song_data[(song_data['track_name'].str.lower() == song_name.lower()) & (song_data['artist_name'].str.lower() == artist_name.lower())]

if not song_query.empty:
    # Scale numerical features of the query
    song_numerical_features = scaler.transform(song_query[numerical_features])
    
    # Encode genre of the query
    song_genre_encoded = encoder.transform(song_query[['genre']])#.toarray()
    
    # Combine scaled numerical features and encoded genre for the query
    song_combined_features = np.hstack((song_numerical_features, song_genre_encoded * 20))
    
    # Use KNN to find the nearest neighbors
    distances, indices = knn.kneighbors(song_combined_features)
    
    print('Closest songs to |', song_name, '-', artist_name, '| are...')
    for i in range(1, 6):
        index = indices[0][i]
        print('----------')
        print(song_data.iloc[index][['track_name', 'artist_name']])
        
else:
    print("No song found. Check the song and artist names.")